In [1]:
import pandas as pd
file_path = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData'

In [2]:
boston = pd.read_csv(file_path + '/clean/boston_weekly_data_with_covariates.csv', low_memory = False)
seattle = pd.read_csv(file_path + '/clean/seattle_weekly_data_with_covariates.csv', low_memory = False)
nyc = pd.read_csv(file_path + '/clean/nyc_weekly_data.csv', low_memory = False)
la = pd.read_csv(file_path + '/clean/la_weekly_data.csv', low_memory = False)

In [3]:
nyc = nyc.rename(columns={"CASERATE": "case_count"})
la = la.rename(columns={"new_case": "case_count"})
boston = boston.rename(columns={"weekly_cases": "case_count"})

In [4]:
nyc['case_count'] = nyc['case_count'].fillna(0)

In [5]:
la = la.drop('caserate', axis=1)

In [6]:
nyc.dtypes

week                 object
ZCTA5CE10             int64
total_count         float64
total_population    float64
crime_rate          float64
e/pop_rate           object
un_rate              object
p_rate               object
in_median            object
in_mean              object
case_count          float64
PCTPOS              float64
week_no             float64
dtype: object

In [23]:
nyc[nyc.isna().any(axis=1)]


,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no
4811,NaT,10020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
11604,NaT,10103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
11605,NaT,10110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
11606,NaT,10111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
11607,NaT,10112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48967,2023-05-14,11430,2.0,332.0,60.240964,39.7,45.0,74.2,NaN,NaN,0.0,0.0,280.0
48968,2023-05-21,11430,1.0,332.0,30.120482,39.7,45.0,74.2,NaN,NaN,0.0,0.0,281.0
48969,2023-05-28,11430,0.0,332.0,0.000000,39.7,45.0,74.2,NaN,NaN,0.0,0.0,282.0
48970,2023-06-04,11430,2.0,332.0,60.240964,39.7,45.0,74.2,NaN,NaN,0.0,0.0,283.0


In [7]:
nyc['e/pop_rate'] = pd.to_numeric(nyc['e/pop_rate'], errors='coerce')
nyc['in_mean'] = pd.to_numeric(nyc['in_mean'], errors='coerce')
nyc['p_rate'] = pd.to_numeric(nyc['p_rate'], errors='coerce')
nyc['un_rate'] = pd.to_numeric(nyc['un_rate'], errors='coerce')
nyc['in_median'] = pd.to_numeric(nyc['in_median'], errors='coerce')

In [8]:
la['e/pop_rate'] = pd.to_numeric(la['e/pop_rate'], errors='coerce')
la['in_mean'] = pd.to_numeric(la['in_mean'], errors='coerce')
la['p_rate'] = pd.to_numeric(la['p_rate'], errors='coerce')
la['un_rate'] = pd.to_numeric(la['un_rate'], errors='coerce')
la['in_median'] = pd.to_numeric(la['in_median'], errors='coerce')

In [9]:
boston['week'] = pd.to_datetime(boston['week'])
seattle['week'] = pd.to_datetime(seattle['week'])
nyc['week'] = pd.to_datetime(nyc['week'])
la['week'] = pd.to_datetime(la['week'])

# 1st Pre-trend Analysis

In [10]:
boston_1st = boston[(boston['week'] >= '2018-01-01') & (boston['week'] <= '2020-06-30')]
seattle_1st = seattle[(seattle['week'] >= '2018-01-01') & (seattle['week'] <= '2020-06-30')]
nyc_1st = nyc[(nyc['week'] >= '2018-01-01') & (nyc['week'] <= '2020-06-30')]
la_1st = la[(la['week'] >= '2018-01-01') & (la['week'] <= '2020-06-30')]

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
boston_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=boston_1st).fit()
boston_1st['y_hat'] = boston_model.predict(boston_1st)
print(boston_1st['y_hat'].mean())
print(boston_model.summary())

27.99919627873173
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     91.42
Date:                Fri, 01 Mar 2024   Prob (F-statistic):           2.18e-91
Time:                        22:53:14   Log-Likelihood:                -18310.
No. Observations:                4026   AIC:                         3.663e+04
Df Residuals:                    4020   BIC:                         3.667e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.5542      2.611  

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/1817823523.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boston_1st['y_hat'] = boston_model.predict(boston_1st)


In [12]:
seattle_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=seattle_1st).fit()
seattle_1st['y_hat'] = seattle_model.predict(seattle_1st)
print(seattle_1st['y_hat'].mean())
print(seattle_model.summary())

17.361528338306698
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     258.3
Date:                Fri, 01 Mar 2024   Prob (F-statistic):          1.43e-235
Time:                        22:53:14   Log-Likelihood:                -14183.
No. Observations:                3510   AIC:                         2.838e+04
Df Residuals:                    3504   BIC:                         2.841e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     27.8073      2.516 

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/3071156503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seattle_1st['y_hat'] = seattle_model.predict(seattle_1st)


In [13]:
nyc_1st.head()

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no
0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,1.0
1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,2.0
2,2018-01-21,10001,120.0,27004.0,44.437861,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,3.0
3,2018-01-28,10001,123.0,27004.0,45.548808,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,4.0
4,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,5.0


In [14]:
nyc_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=nyc_1st).fit()
nyc_1st['y_hat'] = nyc_model.predict(nyc_1st)
print(nyc_1st['y_hat'].mean())
print(nyc_model.summary())

10.162074792167058
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     1791.
Date:                Fri, 01 Mar 2024   Prob (F-statistic):               0.00
Time:                        22:53:14   Log-Likelihood:                -77493.
No. Observations:               23385   AIC:                         1.550e+05
Df Residuals:                   23380   BIC:                         1.550e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.4648      0.293 

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/4163663363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_1st['y_hat'] = nyc_model.predict(nyc_1st)
/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [15]:
la_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=la_1st).fit()
la_1st['y_hat'] = la_model.predict(la_1st)
print(la_1st['y_hat'].mean())
print(la_model.summary())

9.481123269257628
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     959.7
Date:                Fri, 01 Mar 2024   Prob (F-statistic):               0.00
Time:                        22:53:14   Log-Likelihood:                -68088.
No. Observations:               17407   AIC:                         1.362e+05
Df Residuals:                   17401   BIC:                         1.362e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.6452      0.718  

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/2167795371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_1st['y_hat'] = la_model.predict(la_1st)


In [16]:
# nyc_model = smf.ols('crime_rate ~ C(city) + week_no:C(city) + p_rate + un_rate + in_median + case_count', data=nyc_1st).fit()
# nyc_1st['y_hat'] = nyc_model.predict(nyc_1st)
# print(nyc_1st['y_hat'].mean())
# print(nyc_model.summary())

In [17]:
nyc_1st['city'] = 'nyc'
la_1st['city'] = 'la'
boston_1st['city'] = 'boston'
seattle_1st['city'] = 'seattle'

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/765401552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_1st['city'] = 'nyc'
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/765401552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_1st['city'] = 'la'
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_13274/765401552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [18]:
nycla_1st = pd.concat([nyc_1st, la_1st], axis=0)
nycboston_1st = pd.concat([nyc_1st, boston_1st], axis=0)
nycseattle_1st = pd.concat([nyc_1st, seattle_1st], axis=0)

In [21]:
nycla_1st

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no,y_hat,city
0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,1.0,10.926133,nyc
1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,2.0,10.913222,nyc
2,2018-01-21,10001,120.0,27004.0,44.437861,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,3.0,10.900311,nyc
3,2018-01-28,10001,123.0,27004.0,45.548808,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,4.0,10.887400,nyc
4,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,5.0,10.874489,nyc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39206,2020-06-07,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,8721.0,NaN,127.0,4.511086,la
39207,2020-06-14,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,9920.0,NaN,128.0,4.480456,la
39208,2020-06-21,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,14124.0,NaN,129.0,4.405517,la
39209,2020-06-28,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,17174.0,NaN,130.0,4.347594,la


In [22]:
nycla_model = smf.ols('crime_rate ~ C(city) + week_no:C(city) + p_rate + un_rate + in_median + case_count', data=nycla_1st).fit()
nycla_1st['y_hat_nycla'] = nycla_model.predict(nycla_1st)
print(nycla_1st['y_hat_nycla'].mean())
print(nycla_model.summary())

/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1149: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  exog = exog.reindex(exog_index)


ValueError: cannot reindex on an axis with duplicate labels

# 2nd Pre-trend Analysis

In [ ]:
boston_2nd = boston[(boston['week'] >= '2020-07-01') & (boston['week'] <= '2022-04-01')]
seattle_2nd = seattle[(seattle['week'] >= '2020-07-01') & (seattle['week'] <= '2022-04-01')]
nyc_2nd = nyc[(nyc['week'] >= '2020-07-01') & (nyc['week'] <= '2022-04-01')]
la_2nd = la[(la['week'] >= '2020-07-01') & (la['week'] <= '2022-04-01')]

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
boston_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + weekly_cases', data=boston_2nd).fit()
boston_2nd['y_hat'] = boston_model.predict(boston_2nd)
print(boston_2nd['y_hat'].mean())
print(boston_model.summary())

In [ ]:
seattle_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=seattle_2nd).fit()
seattle_2nd['y_hat'] = seattle_model.predict(seattle_2nd)
print(seattle_2nd['y_hat'].mean())
print(seattle_model.summary())

In [ ]:
nyc_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=nyc_2nd).fit()
nyc_2nd['y_hat'] = nyc_model.predict(nyc_2nd)
print(nyc_2nd['y_hat'].mean())
print(nyc_model.summary())

In [ ]:
la_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=la_2nd).fit()
la_2nd['y_hat'] = la_model.predict(la_2nd)
print(la_2nd['y_hat'].mean())
print(la_model.summary())